In [4]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.getcwd()) + '/OpenTraj/opentraj/') # Anaconda python can't find the toolkit path without this for some reason
import numpy as np

### Edinburgh

In [ ]:
from toolkit.loaders.loader_edinburgh import load_edinburgh
opentraj_root = './OpenTraj/'
selected_day = '01Sep' # 3 days of data in total, ['01Jul', '01Aug', '01Sep']
edinburgh_path = os.path.join(opentraj_root, 'datasets/Edinburgh/annotations', 'tracks.%s.txt' % selected_day)
traj_dataset = load_edinburgh(edinburgh_path, title="Edinburgh", 
                              use_kalman=False, scene_id=selected_day, sampling_rate=9)
data = traj_dataset.data

### ETH

In [1]:
### ETH seq

In [107]:
from toolkit.loaders.loader_eth import load_eth
traj_dataset = load_eth("OpenTraj/datasets/ETH/seq_eth/obsmat.txt", sampling_rate=4)
data = traj_dataset.data

num_steps=5

In [108]:
data = data[data['label']=='pedestrian']

In [109]:
agent_ids = data.agent_id.unique()
for agent_id in agent_ids:
    if len(data[data.agent_id == agent_id]) < 2 * num_steps:
        data = data[data.agent_id != agent_id]
agent_ids = data.agent_id.unique()


for agent_id in agent_ids:
    first_x = data[data.agent_id == agent_id]['pos_x'].iloc[0]
    first_y = data[data.agent_id == agent_id]['pos_y'].iloc[0]

    # 'Normalize' the data so that all trajectories will begin at x=0, y=0
    data.loc[data.agent_id == agent_id, 'pos_x'] = data[data.agent_id == agent_id]['pos_x'] - first_x
    data.loc[data.agent_id == agent_id, 'pos_y'] = data[data.agent_id == agent_id]['pos_y'] - first_y
    # Calculate own velocities so that they don't depend on sampling rate
    data.loc[data.agent_id == agent_id, 'vel_x'] = data[data.agent_id == agent_id]['pos_x'].diff()
    data.loc[data.agent_id == agent_id, 'vel_y'] = data[data.agent_id == agent_id]['pos_y'].diff()

In [2]:
### seq_hotel

In [8]:
from toolkit.loaders.loader_eth import load_eth
traj_dataset = load_eth("OpenTraj/datasets/ETH/seq_hotel/obsmat.txt", sampling_rate=4)
print(traj_dataset.data)

num_steps=5

Empty DataFrame
Columns: [index, frame_id, agent_id, pos_x, pos_y, vel_x, vel_y, scene_id, label, timestamp]
Index: []


In [6]:
print(data)

Empty DataFrame
Columns: [index, frame_id, agent_id, pos_x, pos_y, vel_x, vel_y, scene_id, label, timestamp]
Index: []


### UCY

In [10]:
import os
from toolkit.loaders.loader_crowds import load_crowds
# fixme: replace OPENTRAJ_ROOT with the address to root folder of OpenTraj
OPENTRAJ_ROOT = './OpenTraj/'
zara01_annot = os.path.join(OPENTRAJ_ROOT, 'datasets/UCY/zara01/annotation.vsp')
zara01_H_file = os.path.join(OPENTRAJ_ROOT, 'datasets/UCY/zara01/H.txt')
traj_dataset = load_crowds(zara01_annot, use_kalman=False, homog_file=zara01_H_file)

In [11]:
print(traj_dataset.data)

      frame_id  agent_id      pos_x     pos_y  scene_id       label  \
0           10         0   1.230339  2.622447         0  pedestrian   
1           20         0   1.863678  2.649175         0  pedestrian   
2           30         0   2.483173  2.693372         0  pedestrian   
3           40         0   3.088825  2.755038         0  pedestrian   
4           50         0   3.694478  2.816704         0  pedestrian   
...        ...       ...        ...       ...       ...         ...   
4981      8970       147  15.744374  5.688536         0  pedestrian   
4982      8980       147  15.860835  5.635239         0  pedestrian   
4983      8990       147  16.211599  5.474385         0  pedestrian   
4984      9000       147  16.562362  5.313531         0  pedestrian   
4985      9010       147  16.913126  5.152677         0  pedestrian   

      timestamp     vel_x     vel_y  
0           0.4  1.583348  0.066819  
1           0.8  1.583348  0.066819  
2           1.2  1.548739  0.1104

### HERMES

In [26]:
import os
from toolkit.loaders.loader_hermes import load_bottleneck
serie = 'Corridor-2D'
exp = 'bo-360-160-160'
OPENTRAJ_ROOT = './OpenTraj/'
annot_file = os.path.join(OPENTRAJ_ROOT, "datasets/HERMES", serie, exp + '.txt')
traj_datasets = load_bottleneck(annot_file, sampling_rate=6, use_kalman=False, title='Hermes')

In [27]:
print(traj_datasets.data)

       index  frame_id  agent_id    pos_x    pos_y  scene_id       label  \
0          1        66         1  6.66089  2.29151         0  pedestrian   
1          7        72         1  6.19305  2.28999         0  pedestrian   
2         13        78         1  5.70179  2.32027         0  pedestrian   
3         19        84         1  5.22111  2.28289         0  pedestrian   
4         25        90         1  4.70377  2.28232         0  pedestrian   
...      ...       ...       ...      ...      ...       ...         ...   
15365  92175       582       309  5.44977  3.55281         0  pedestrian   
15366  92181       588       309  5.77591  3.78994         0  pedestrian   
15367  92187       594       309  6.11509  4.00961         0  pedestrian   
15368  92193       600       309  6.35927  4.31885         0  pedestrian   
15369  92199       606       309  6.55438  4.70483         0  pedestrian   

       timestamp     vel_x     vel_y  
0          4.125 -1.247573 -0.004053  
1        